# Importing dependencies

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

### important
Dont forget to install necessary libraries

```
pip install tensorflow keras numpy matplotlib ipython
```

### usage of libraries
**tensorflow** - various machine learning tasks, especially deep learning<br>
**keras** - part of tensorflow, Python interface for artificial neural networks<br>
**numpy** - fast scientific/numerical calculations<br>
**matplotlib** - visual representations of data<br>
**IPython** - utilities to display various media representations in Jupyter notebooks<br>

# Loading data into tensorflow dataset object

In [6]:
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)

In [7]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    batch_size = BATCH_SIZE,
    image_size = IMAGE_SIZE,
    shuffle = True,
    seed = 24
)

Found 16011 files belonging to 10 classes.


# What is tensorflow Dataset?

**tf Dataset** is a powerful framework designed to handle large datasets efficiently by loading data in batches and providing a lot of functions for preprocessing and data transformation, like filtering, mapping etc. 

In [8]:
class_names = dataset.class_names
class_names

['Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']